Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos <br>
Segundo Semestre 2024
<br>

<h1><center>Tarea 2: Boosting </center></h1>
        Fecha de entrega: 13 de septiembre de 2024 a las 20:00 hrs

# 1.1 Enunciado

La meta de esta tarea es intentar subir el 0.48 de accuracy que nos dio el método de árboles de decisión para el caso de los helados. Como ya vimos que con ensamblajes ya no funcionó, vamos a usar **boosting**.



**IMPORTANTE**:

- Esta tarea es **individual**, se entrega como un notebook con todas las celdas corridas. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

- La fecha de entrega de la tarea es el Viernes 13 de Septiembre, a las 20:00 hrs.

---


## Librerías

In [1]:
# Aqui puedes cargar las librerias a utilizar
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score



# Cargando datos

In [2]:
helados = pd.read_csv('Ice_cream.csv')
X = helados[['female', 'video', 'puzzle']]
y = helados['ice_cream']
helados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  200 non-null    int64
 1   id          200 non-null    int64
 2   female      200 non-null    int64
 3   ice_cream   200 non-null    int64
 4   video       200 non-null    int64
 5   puzzle      200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB


# 1.2. Primer boosting

### 1.2.1  Genera un train/test split con un 30 % de datos de test y un 70 % en el entrenamiento.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### 1.2.2  Entrena un clasificador de árbol de decisión sobre los datos de entrenamiento como lo vimos en clases, llamémoslo **clf**.

Comprueba que el accuracy para los datos de test sigue siendo 0.48 o menor.

In [4]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
prediction = clf.predict(X_test)


In [5]:
first_accuracy = accuracy_score(prediction, y_test)
print(f"El accuracy de mi modelo es: {first_accuracy:.2f}")

El accuracy de mi modelo es: 0.56


### 1.2.3  Recuerda que podemos usar **clf.predict proba** para obtener las probabilidades de clasificar cada instancia.

Define entonces el error *e* como *e[i] = y[i] - proba[i]*, donde y es el vector que contiene las clases correctas de los datos de entrenamiento, y proba[i] es la segunda componente del i-esimo elemento del vector que resulta al llamar a **clf.predict proba** con los datos de entrenamiento.

In [6]:
# Predecir las probabilidades para los datos de entrenamiento
probabilidad = clf.predict_proba(X_train)

# Extraer la segunda componente de cada elemento de probas (probabilidad de clase 1)
probabilidad = probabilidad[:, 1]

# Calcular el error e
error = y_train - probabilidad

error

103   -0.500000
153    0.666667
54     0.000000
72     0.500000
179    0.000000
         ...   
192    0.000000
10     0.000000
82     0.000000
127    0.000000
168    0.000000
Name: ice_cream, Length: 150, dtype: float64

### 1.2.4 Entrena un árbol de regresión.

Usando la clase **DecisionTreeRegressor** de **sklearn.tree**, de forma
que pueda predecir el vector *e* a partir de los datos de entrenamiento. Llamemoslo **clf boost**

In [7]:
clf_boost = DecisionTreeRegressor()
clf_boost.fit(X_train, error)

DecisionTreeRegressor()

### 1.2.5 Define un nuevo clasificador que clasifique a una entrada como un 1 si el promedio del resultado de **clf** y clf boost es mayor o igual a 0.5, y como 0 en otro caso.

In [8]:
def clasificador(clf, clf_boost, X_test):
    prediction = clf.predict(X_test)
    prediction_error = clf_boost.predict(X_test)

    avg_prediction = (prediction + prediction_error) / 2

    return (avg_prediction >= 0.5).astype(int)

In [9]:
new_predictions = clasificador(clf, clf_boost, X_test)
new_predictions

array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1])

### 1.2.6 Prueba el accuracy de este clasificador al usar el set de test. ¿Es mejor?

In [17]:
new_accuracy = accuracy_score(y_test, new_predictions)
print(new_accuracy)
def tester(first_accuracy, new_accuracy):
    if first_accuracy > new_accuracy:
        print("El primer test es mejor")
    elif new_accuracy > first_accuracy:
        print("El nuevo test es mejor")
    else: 
        print(f"Son iguales")

tester(first_accuracy, new_accuracy)

0.56
Son iguales


# 1.3. Segundo boosting

#### Vuelve a entrenar ahora un tercer clasificador, **DecisionTreeRegressor**, ahora sobre el error definido como la diferencia entre *e* y el resultado de **clf boost**, tal y como lo hiciste en la sección anterior. Verifica si logras subir o no el accuracy más allá de lo logrado en la sección anterior.

In [11]:
# Predecir las probabilidades para los datos de entrenamiento
probabilidad_2 = clf.predict_proba(X_train)

# Extraer la segunda componente de cada elemento de probas (probabilidad de clase 1)
probabilidad_2 = probabilidad_2[:, 1]

# Calcular el error e
error_2 = error - probabilidad_2

error_2

103   -1.000000
153    0.333333
54    -1.000000
72     0.000000
179    0.000000
         ...   
192   -1.000000
10    -1.000000
82    -1.000000
127   -1.000000
168   -1.000000
Name: ice_cream, Length: 150, dtype: float64

In [12]:
clf_boost_2 = DecisionTreeRegressor()
clf_boost_2.fit(X_train, error)

new_new_predictions = clasificador(clf, clf_boost_2, X_test)
new_new_predictions

array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 1])

In [13]:
ac_pred = accuracy_score(prediction, new_new_predictions)
tester(first_accuracy, ac_pred)

El nuevo test es mejor


# 1.4. ¡Cuidado con las métricas!

#### Mira ahora el score f1 de lo que has hecho. Examina un poco mejor para ver por qué (en este caso) el boosting resultó en un mal clasificador.



In [24]:

from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision, recall, and F1 score for each class
class_labels = [0,1]

In [26]:
print("Para el predictor original:\n")

for label in class_labels:
    print(f"Class: {label}")
    print(f"Precision: {precision_score(y_test, prediction, pos_label=label)}")
    print(f"Recall: {recall_score(y_test, prediction, pos_label=label)}")
    print(f"F1 score: {f1_score(y_test, prediction, pos_label=label)}\n")

Para el predictor original:

Class: 0
Precision: 0.6190476190476191
Recall: 0.48148148148148145
F1 score: 0.5416666666666666

Class: 1
Precision: 0.5172413793103449
Recall: 0.6521739130434783
F1 score: 0.5769230769230769



In [27]:
print("Para la primera iteración:\n")

for label in class_labels:
    print(f"Class: {label}")
    print(f"Precision: {precision_score(y_test, new_predictions, pos_label=label)}")
    print(f"Recall: {recall_score(y_test, new_predictions, pos_label=label)}")
    print(f"F1 score: {f1_score(y_test, new_predictions, pos_label=label)}\n")

Para la primera iteración:

Class: 0
Precision: 0.6190476190476191
Recall: 0.48148148148148145
F1 score: 0.5416666666666666

Class: 1
Precision: 0.5172413793103449
Recall: 0.6521739130434783
F1 score: 0.5769230769230769



In [28]:
print("Para la segunda iteración:\n")

for label in class_labels:
    print(f"Class: {label}")
    print(f"Precision: {precision_score(y_test, new_new_predictions, pos_label=label)}")
    print(f"Recall: {recall_score(y_test, new_new_predictions, pos_label=label)}")
    print(f"F1 score: {f1_score(y_test, new_new_predictions, pos_label=label)}\n")

Para la segunda iteración:

Class: 0
Precision: 0.6190476190476191
Recall: 0.48148148148148145
F1 score: 0.5416666666666666

Class: 1
Precision: 0.5172413793103449
Recall: 0.6521739130434783
F1 score: 0.5769230769230769

